<table style="border:none; border-collapse:collapse; cellspacing:0; cellpadding:0">
<tr>
    <td width=30% style="border:none">
        <center>
            <img src="../images/iapau_icon.png" width="30%"/><br>
            <a href="https://iapau.org/">Association IA Pau</a><br>
            <a href="https://iapau.org/events/festival/">Festival IAPau 7</a>
        </center>
    </td>
    <td style="border:none">
        <center>
            <h1>Atelier - Agentic RAG</h1>
            <h2>Agents Spécialisés (Outils)</h2>
            <h2>librarian - Analyst - Advanced Analyst - Scout</h2>
        </center>
    </td>
    <td width=20% style="border:none">
    </td>
</tr>
</table>

---

**Prérequis :** Compléter d'abord les Phases 0 et 1.

---

## 📋 Table des matières

- [**Import des bibliothèques et chargement des données**](#import-libraries-and-load-data-from-phase-1)

1. [**Agent `librarian` (Outil RAG avancé)**](#librarian-agent)
   - Optimisation de requêtes avec LLM
   - Recherche vectorielle avec re-classement par encodeur croisé
   - Recherche dans les documents financiers

2. [**Agent `Analyst` (Outil SQL)**](#analyst-agent)
   - Requêtes SQL directes sur la base de données financières
   - Agent SQL LangChain pour des données spécifiques

3. [**Agent `Advanced Analyst` (Outil d'analyse des tendances)**](#advanced-analyst)
   - Analyse de tendances multi-périodes
   - Calcul des taux de croissance QoQ et YoY
   - Génération de résumés narratifs

4. [**Agent `Scout` (Outil de recherche web)**](#scout-agent)
   - Recherche web en temps réel avec Tavily
   - Accès aux informations actuelles et externes

- [**Prochaines étapes**](#phase-2-terminée)
   - Récapitulatif des outils créés
   - Préparation pour l'orchestration (Phase 3)

---

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import sqlite3
import json
from typing import List, Dict, Any

from langchain_openai import ChatOpenAI
from langchain.tools import tool
from sentence_transformers import CrossEncoder
from fastembed import TextEmbedding
import qdrant_client
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.tools.tavily_search import TavilySearchResults

from IPython.display import display, Markdown, HTML
import ipywidgets as widgets

In [2]:
# Load enriched chunks from Phase 1
with open('enriched_chunks.json', 'r') as f:
    all_enriched_chunks = json.load(f)
print(f"Loaded {len(all_enriched_chunks)} enriched chunks")

# Initialize embedding model
embedding_model = TextEmbedding(model_name="BAAI/bge-small-en-v1.5")
print("Embedding model initialized")

# Connect to existing Qdrant collection
QDRANT_PATH = "./qdrant_storage"
COLLECTION_NAME = "financial_docs"

client = qdrant_client.QdrantClient(path=QDRANT_PATH)
collection_info = client.get_collection(collection_name=COLLECTION_NAME)
print(f"Loaded Qdrant collection with {collection_info.points_count} points")

# Load SQL database
DB_PATH = "financials.db"
db = SQLDatabase.from_uri(f"sqlite:///{DB_PATH}")
print("SQL database loaded")

Loaded 566 enriched chunks


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

Embedding model initialized
Loaded Qdrant collection with 566 points
SQL database loaded


<a id="librarian-agent"></a>
### <b><div style='padding:15px;background-color:#4A5568;color:white;border-radius:2px;font-size:110%;text-align: left'>1. `Librarian` Agent - Outil RAG avancé</div></b>

<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0f7ff,#ffffff);padding:16px;border-left:6px solid #2b6cb0;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>🚀</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Ce que nous allons faire</h2>
    <p style='margin:0 0 8px 0;color:#000;'>Notre outil/agent <i>Librarian</i> est la pierre angulaire pour la récupération de nos données non structurées. Son processus en plusieurs étapes (cf. ci-dessous) fournira le contexte nécessaire à tous les raisonnements en aval.<br><br><strong>Optimiseur de requête -> Recherche vectorielle -> Re-classement</strong></p>
  </div>
</div>

In [4]:
query_optimizer_llm = ChatOpenAI(
    model='gpt-4o-mini',
    api_key=os.getenv('OPENAI_API_KEY'),
    temperature=0.
)

def optimize_query(query: str) -> str:
    """Uses an LLM to rewrite a query for better retrieval."""
    prompt = f"""
    You are a query optimization expert. Rewrite the following user query to be more specific and effective for searching through corporate financial documents.
    
    User Query: {query}
    
    Return ONLY the optimized query text with no labels, explanations, or additional formatting.
    """
    optimized_query = query_optimizer_llm.invoke(prompt).content
    return optimized_query

# Test
original_query = "Comment se porte Nvidia dans ses activités en Europe ?" #"How is Nvidia doing with its Europe business?"
optimized_query_result = optimize_query(original_query)

print(f"Original Query: {original_query}")
print(f"\nOptimized Query: {optimized_query_result}")

Original Query: Comment se porte Nvidia dans ses activités en Europe ?

Optimized Query: Analyse des performances financières de Nvidia en Europe au cours des dernières années.


<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0fff4,#ffffff);padding:16px;border-left:6px solid #16a34a;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>✅</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Discussion de la sortie</h2>
    <p style='margin:0 0 8px 0;color:#000;'>La requête originale, assez vague, a été transformée en une requête beaucoup plus précise, incluant des termes spécifiques. Cette requête améliorée conduira à de bien meilleurs résultats.</p>
  </div>
</div>

In [5]:
cross_encoder_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

@tool
def librarian_rag_tool(query: str, verbose: bool = False) -> List[Dict[str, Any]]:
    """ 
    Expert at finding information from Nvidia's financial documents.
    Use for questions about financial performance, business segments, products, risks, strategies.
    
    Args:
        query: The search query
        verbose: If True, prints detailed execution steps (default: False)
    """
    from IPython.display import display, Markdown
    
    if verbose:
        display(Markdown(f"""
        ---
        ### 🔍 Librarian Tool - Exécution détaillée

        **📝 Requête originale:** `{query}`
        """))
    
    # Step 1: Optimize query for better retrieval
    optimized_query = optimize_query(query)
    if verbose:
        display(Markdown(f"""
        **Étape 1/4: Optimisation de la requête** ✨

        - **Requête optimisée:** `{optimized_query}`
        - **Action:** Réécriture de la requête avec le LLM pour améliorer la recherche
        """))
    
    # Step 2: Perform vector search
    query_embedding = list(embedding_model.embed([optimized_query]))[0]
    search_results = client.search(
        collection_name=COLLECTION_NAME,
        query_vector=query_embedding.tolist(),
        limit=20,
        with_payload=True
    )
    if verbose:
        display(Markdown(f"""
        **Étape 2/4: Recherche vectorielle** 🔎

        - **Embedding généré:** Vecteur de dimension {len(query_embedding)}
        - **Collection Qdrant:** `{COLLECTION_NAME}`
        - **Candidats récupérés:** {len(search_results)} documents
        - **Méthode:** Similarité cosinus
        """))
    
    # Step 3: Re-rank results using cross-encoder
    rerank_pairs = [[optimized_query, result.payload['content']] for result in search_results]
    scores = cross_encoder_model.predict(rerank_pairs)
    
    # Update scores and sort
    for result, score in zip(search_results, scores):
        result.score = score
    reranked_results = sorted(search_results, key=lambda x: x.score, reverse=True)
    
    if verbose:
        display(Markdown(f"""
        **Étape 3/4: Re-classement avec Cross-Encoder** 🎯

        - **Modèle:** ms-marco-MiniLM-L-6-v2
        - **Documents re-classés:** {len(search_results)}
        - **Top 3 scores:**
            - 1er: {reranked_results[0].score:.4f}
            - 2ème: {reranked_results[1].score:.4f}
            - 3ème: {reranked_results[2].score:.4f}
        """))
    
    # Step 4: Extract and return top results
    top_k = 5
    final_results = [
        {
            'source': result.payload['source'],
            'content': result.payload['content'],
            'summary': result.payload['summary'],
            'rerank_score': float(result.score)
        }
        for result in reranked_results[:top_k]
    ]
    
    if verbose:
        display(Markdown(f"""
        **Étape 4/4: Sélection finale** ✅

        - **Documents retournés:** {top_k} meilleurs résultats
        - **Score moyen:** {sum(r['rerank_score'] for r in final_results) / len(final_results):.4f}
        - **Sources:** {len(set(r['source'] for r in final_results))} documents distincts

        ---
        """))
    
    return final_results

In [9]:
# Test 
test_query = "What are the main risks associated with chip production in the field of AI?"
#test_query = "Quels sont les principaux risques associés à la poduction des puces dans le domaine de l'IA ?" 


# Create toggle for verbose mode
verbose_toggle = widgets.Checkbox(
    value=False,
    description='Afficher le détail des étapes',
    style={'description_width': 'initial'}
)

# Create button to run test
run_button = widgets.Button(
    description='🔍 Lancer le test',
    button_style='primary',
    tooltip='Exécuter la recherche'
)

output_area = widgets.Output()

def on_button_click(b):
    with output_area:
        output_area.clear_output()
        
        # Run the tool with verbose option
        librarian_results = librarian_rag_tool.invoke({
            "query": test_query,
            "verbose": verbose_toggle.value
        })
        
        # Display with beautiful formatted output
        display(HTML("""
<style>
.result-card {
    border-left: 4px solid #4A90E2;
    padding: 15px;
    margin: 10px 0;
    background: linear-gradient(to right, #f8f9fa, #ffffff);
    border-radius: 5px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
}
.score-badge {
    background: #4A90E2;
    color: white;
    padding: 3px 10px;
    border-radius: 12px;
    font-size: 0.85em;
    font-weight: bold;
}
.source-tag {
    background: #f0f0f0;
    padding: 2px 8px;
    border-radius: 3px;
    font-family: monospace;
    font-size: 0.9em;
}
</style>
"""))
        
        display(Markdown(f"""
---
## 🔍 Test du Librarian Tool

<div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; color: white; margin: 20px 0;'>
<h3 style='margin:0; color: white;'>📌 Requête</h3>
<p style='font-size: 1.1em; font-style: italic; margin: 10px 0 0 0;'>{test_query}</p>
</div>

### ✨ Top 3 résultats les plus pertinents ({len(librarian_results)} trouvés au total)
"""))
        
        # Show only top 3 results
        for i, result in enumerate(librarian_results[:3], 1):
            colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
            color = colors[i-1]
            
            display(HTML(f"""
    <div class='result-card' style='border-left-color: {color};'>
        <h4 style='color: {color}; margin-top: 0;'>📄 Résultat #{i}</h4>
        <p><strong>Source:</strong> <span class='source-tag'>{result['source']}</span> 
        <span class='score-badge'>Score: {result['rerank_score']:.4f}</span></p>
    </div>
    """))
            
            display(Markdown(f"""
**💡 Résumé:**
> {result['summary']}

<details>
<summary><b>📖 Voir le contenu complet (cliquez pour développer)</b></summary>

```text
{result['content'][:800]}...
```
</details>
"""))
            
            if i < 3:
                display(HTML("<hr style='border: none; border-top: 2px dashed #e0e0e0; margin: 20px 0;'>"))
        
        display(Markdown("""
---
### ✅ Test complété avec succès!
"""))

run_button.on_click(on_button_click)

# Display controls
display(Markdown("### ⚙️ Configuration du test"))
display(widgets.HBox([verbose_toggle, run_button]))
display(output_area)

### ⚙️ Configuration du test

Output()

<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0fff4,#ffffff);padding:16px;border-left:6px solid #16a34a;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>✅</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Discussion de la sortie</h2>
    <p style='margin:0 0 8px 0;color:#000;'>Le journal d'exécution montre chaque étape : l'appel de l'outil, l'optimisation de la requête, la récupération initiale, le re-classement, et la sélection finale des 5 meilleurs résultats. La sortie finale (JSON) est une liste des fragments de texte les plus pertinents de notre base de données non-structurées (SEC), chacun avec son document source et son score de re-classement. On peut vérifier que le contenu de ces fragments est hautement pertinent. Le <i>Librarian</i> est prêt.</p>
  </div>
</div>

<a id="analyst-agent"></a>
### <b><div style='padding:15px;background-color:#4A5568;color:white;border-radius:2px;font-size:110%;text-align: left'>2. `Analyst` Agent (Outil SQL)</div></b>

<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0f7ff,#ffffff);padding:16px;border-left:6px solid #2b6cb0;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>🚀</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Ce que nous allons faire</h2>
    <p style='margin:0 0 8px 0;color:#000;'>L'outil <i>Analyst</i> sert à interroger des données spécifiques, ponctuelles, de notre base de données structurée. Son rôle est bien ciblé. Les questions plus avancées sur les tendances seront traitées par notre outil <i>Advanced Analyst</i>. Une bonne description des outils qui clarifie cette distinction permettra au <i>Supervisor</i> de bien choisir les outils dont il à besoin.</p>
  </div>
</div>

In [10]:
sql_agent_llm = ChatOpenAI(model='gpt-4o', api_key=os.getenv('OPENAI_API_KEY'), temperature=0.)

sql_agent_executor = create_sql_agent(llm=sql_agent_llm, db=db, agent_type="openai-tools", verbose=False)

@tool
def analyst_sql_tool(query: str, verbose: bool = False) -> str:
    """
    Expert financial analyst that queries Nvidia's revenue and net income database.
    Use for questions about specific financial numbers for a single time period.
    For trends over time, use the analyst_trend_tool.
    Args:
        query: The SQL query
        verbose: If True, prints detailed execution steps
    """
    from IPython.display import display, Markdown
    if verbose:
        display(Markdown(f"""
---
### 🧑‍💼 Analyst SQL Tool - Exécution détaillée

**📝 Requête originale:** `{query}`
"""))
    print(f"\n-- Analyst SQL Tool Called with query: '{query}' --")
    result = sql_agent_executor.invoke({"input": query})
    if verbose:
        display(Markdown(f"""
**Étape 1/2: Exécution SQL** 🗄️

- **Requête SQL générée:**
```sql
{result.get('intermediate_steps', '')}
```
- **Résultat brut:** {result.get('output', '')}
"""))
    return result['output']


In [16]:
# Test interactif avec option verbose et affichage de la requête SQL générée
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML

test_sql_query = "What was the total revenue for the full year 2023?"
verbose_sql_toggle = widgets.Checkbox(
    value=False,
    description='Afficher le détail des étapes',
    style={'description_width': 'initial'}
)
run_sql_button = widgets.Button(
    description='🧑‍💼 Lancer le test Analyst',
    button_style='primary',
    tooltip='Exécuter la requête SQL'
)
output_sql_area = widgets.Output()

def on_sql_button_click(b):
    with output_sql_area:
        output_sql_area.clear_output()
        result = sql_agent_executor.invoke({"input": test_sql_query})
        analyst_result = analyst_sql_tool.invoke({
            "query": test_sql_query,
            "verbose": verbose_sql_toggle.value
        })
        display(HTML("""
<style>
.analyst-result-card {
    background: linear-gradient(90deg, #f8fafc 0%, #e3e8ee 100%);
    border-left: 6px solid #2b6cb0;
    padding: 18px;
    margin: 18px 0;
    border-radius: 8px;
    box-shadow: 0 2px 8px rgba(44,62,80,0.07);
    color: #22223b;
    font-size: 1.08em;
}
pre.analyst-output {
    background: #22223b;
    color: #f2e9e4;
    padding: 12px;
    border-radius: 6px;
    font-size: 1em;
    overflow-x: auto;
}
pre.analyst-sql {
    background: #e2e8f0;
    color: #2b6cb0;
    padding: 10px;
    border-radius: 6px;
    font-size: 1em;
    margin-bottom: 10px;
    font-family: 'Fira Mono', 'Consolas', monospace;
}
</style>
"""))
        display(Markdown("""
---
## 🧑‍💼 Test du Analyst Tool

<div class='analyst-result-card'>
<b>Requête :</b> <span style='color:#2b6cb0;font-weight:bold'>{test_sql_query}</span>
</div>

<b>Requête SQL générée :</b>
"""))
        sql_steps = result.get('intermediate_steps', '')
        if sql_steps:
            display(HTML(f"<pre class='analyst-sql'>{sql_steps}</pre>"))
        else:
            display(Markdown("> *Aucune requête SQL générée visible.*"))
        display(Markdown("<b>Résultat final :</b>"))
        display(HTML(f"<pre class='analyst-output'>{analyst_result}</pre>"))
        display(Markdown("---"))

run_sql_button.on_click(on_sql_button_click)
display(Markdown("### ⚙️ Configuration du test Analyst"))
display(widgets.HBox([verbose_sql_toggle, run_sql_button]))
display(output_sql_area)

### ⚙️ Configuration du test Analyst

Output()

<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0fff4,#ffffff);padding:16px;border-left:6px solid #16a34a;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>✅</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Discussion de la sortie</h2>
    <p style='margin:0 0 8px 0;color:#000;'>Nous pouvons voir toute la chaîne de pensée du LLM. Il inspecte les tables, formule la requête SQL correcte (<code>SELECT sum(revenue_usd_billions) FROM revenue_summary WHERE year = 2023</code>), l'exécute, obtient le résultat numérique, puis formule une réponse en langage naturel. La sortie finale est la réponse claire et correcte à notre question. L'<i>Analyst</i> est opérationnel.</p>
  </div>
</div>

<a id="advanced-analyst"></a>
### <b><div style='padding:15px;background-color:#4A5568;color:white;border-radius:2px;font-size:110%;text-align: left'>3. `Advanced Analyst` Agent (Outil d'analyse des tendances)</div></b>

<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0f7ff,#ffffff);padding:16px;border-left:6px solid #2b6cb0;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>🚀</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Ce que nous allons faire</h2>
    <p style='margin:0 0 8px 0;color:#000;'>Un analyste humain examine rarement un chiffre de manière isolée; il analyse les tendances. Nous allons construire un nouvel outil, <code>analyst_trend_tool</code>, qui va au-delà de la simple récupération SQL. Il va récupérer des données sur plusieurs périodes, calculer des métriques clés comme la croissance trimestre sur trimestre (QoQ) et année sur année (YoY), et fournir un résumé narratif. Cela fait passer l'agent d'un simple récupérateur de données à un *interpréteur* de données.</p>
  </div>
</div>

In [18]:
@tool
def analyst_trend_tool(query: str) -> str:
    """
    Analyzes financial data over multiple time periods to identify trends and growth rates.
    Best for questions like 'Analyze revenue trend' or 'Show me growth YoY'.
    Provides narrative summary of trends, not just raw numbers.
    """
    print(f"\n-- Analyst Trend Tool Called with query: '{query}' --")
    
    conn = sqlite3.connect(DB_PATH)
    df_trends = pd.read_sql_query("SELECT * FROM revenue_summary ORDER BY year, quarter", conn)
    conn.close()

    df_trends['period'] = df_trends['year'].astype(str) + '-' + df_trends['quarter']
    df_trends.set_index('period', inplace=True)
    
    metric = 'revenue_usd_billions'
    df_trends['QoQ_Growth'] = df_trends[metric].pct_change()
    df_trends['YoY_Growth'] = df_trends[metric].pct_change(4)
    
    latest_period = df_trends.index[-1]
    start_period = df_trends.index[0]
    latest_val = df_trends.loc[latest_period, metric]
    start_val = df_trends.loc[start_period, metric]
    latest_qoq = df_trends.loc[latest_period, 'QoQ_Growth']
    latest_yoy = df_trends.loc[latest_period, 'YoY_Growth']
    
    summary = f"""
    Analysis of {metric} from {start_period} to {latest_period}:
    - The series shows a general upward trend, starting at ${start_val}B and ending at ${latest_val}B.
    - The most recent quarter ({latest_period}) had a Quarter-over-Quarter growth of {latest_qoq:.1%}.
    - The Year-over-Year growth for the most recent quarter was {latest_yoy:.1%}.
    - Overall, performance indicates consistent growth over the analyzed period.
    """
    return summary

In [19]:
# Test
trend_result = analyst_trend_tool.invoke("Analyze the revenue trend over the last two years")
print("\n--- Analyst Trend Tool Output ---")
print(trend_result)


-- Analyst Trend Tool Called with query: 'Analyze the revenue trend over the last two years' --

--- Analyst Trend Tool Output ---

    Analysis of revenue_usd_billions from 2020-Q1 to 2024-Q4:
    - The series shows a general upward trend, starting at $3.11B and ending at $35.08B.
    - The most recent quarter (2024-Q4) had a Quarter-over-Quarter growth of 16.8%.
    - The Year-over-Year growth for the most recent quarter was 93.6%.
    - Overall, performance indicates consistent growth over the analyzed period.
    


<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0fff4,#ffffff);padding:16px;border-left:6px solid #16a34a;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>✅</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Discussion de la sortie</h2>
    <p style='margin:0 0 8px 0;color:#000;'>Au lieu d'une simple liste de chiffres, l'outil a produit un résumé lisible qui capture l'essence de l'histoire des données. Il fournit les valeurs de début et de fin pour le contexte et calcule les métriques les plus importantes (croissance QoQ et YoY). L'agent <i>Supervisor</i> dispose maintenant d'un outil puissant pour comprendre non seulement <strong>quels sont</strong> les chiffres, mais <strong>ce que signifient les chiffres</strong> en termes de performance dans le temps.</p>
  </div>
</div>

<a id="scout-agent"></a>
### <b><div style='padding:15px;background-color:#4A5568;color:white;border-radius:2px;font-size:110%;text-align: left'>4. `Scout` Agent (Web Search Tool)</div></b>

<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0f7ff,#ffffff);padding:16px;border-left:6px solid #2b6cb0;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>🚀</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Ce que nous allons faire</h2>
    <p style='margin:0 0 8px 0;color:#000;'>Le <i>Scout</i> fournira à l'agent un accès à Internet en direct via l'API de recherche <i>Tavily</i>. C'est crucial pour répondre aux questions sur les événements en temps réel, comme les cours des actions ou les actualités récentes, qui ne sont pas présents dans les dépôts SEC statiques.</p>
  </div>
</div>

In [ ]:
scout_web_search_tool = TavilySearchResults(max_results=3)
scout_web_search_tool.name = "scout_web_search_tool"
scout_web_search_tool.description = (
    "Web search expert. Use to find real-time information not in financial documents, "
    "such as current stock prices, recent news, or competitor information."
)

print("Scout tool is configured.")

# Test
test_web_query = "What is the current stock price of Microsoft (MSFT)?"
print(f"\n-- Scout Tool Called with query: '{test_web_query}' --")
scout_result = scout_web_search_tool.invoke({"query": test_web_query})

print("\n--- Scout Tool Output ---")
print(json.dumps(scout_result[:2], indent=2))  # Show first 2 results

######### UPDATE

scout_web_search_tool = TavilySearchResults(max_results=3)
# The tool is already created by the library, we just give it a new name and description for our Supervisor
scout_web_search_tool.name = "scout_web_search_tool"
scout_web_search_tool.description = ( 
    "This tool is a web search expert. Use it to find real-time information that is not available in the financial documents, "
    "such as current stock prices, recent news, or information about competitor companies." 
)

print("Scout tool is configured.")

# --- Test the Scout Tool ---
test_web_query = "What is the current stock price of Nvidia (NVDA)?"
print(f"\n-- Scout Tool Called with query: '{test_web_query}' --")
scout_result = scout_web_search_tool.invoke({"query": test_web_query})

print("\n--- Scout Tool Output ---")
print(json.dumps(scout_result, indent=2))

<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0fff4,#ffffff);padding:16px;border-left:6px solid #16a34a;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>✅</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Discussion de la sortie</h2>
    <p style='margin:0 0 8px 0;color:#000;'>L'outil <i>Scout</i> a interrogé avec succès l'API <i>Tavily</i> et a renvoyé une liste concise de résultats de recherche, incluant des extraits de pages web pertinentes. Cela démontre sa capacité à récupérer des données en temps réel depuis Internet.</p> 
    <p style='margin:0 0 8px 0;color:#000;'>Avec le <i>Librarian</i>, l'<i>Analyst</i>, l'<i>Advanced Analyst</i> et le <i>Scout</i>, notre équipe de spécialistes est complète. Il est temps de construire notre <i>Supervisor</i> pour les gérer et les faire travailler correctement ensemble.</p>
  </div>
</div>

---

<a id="phase-2-terminée"></a>
### Prochaines étapes

**Ce que nous avons construit :**
- **Outil Librarian** : RAG avancé avec optimisation de requête et re-classement par encodeur croisé
- **Outil Analyst SQL** : Requêtes SQL directes pour des points de données spécifiques
- **Outil Analyst Trend** : Analyse multi-périodes avec calcul des métriques de croissance
- **Outil Scout** : Capacités de recherche web en temps réel

**Variables clés pour la Phase 3 :**
- `tools` - Liste de tous les outils d'agent
- `tool_map` - Dictionnaire associant les noms d'outils aux fonctions d'outils
- `embedding_model`, `client`, `db` - Ressources chargées

<a href="./phase_3_reasoning_engine.ipynb" style="font-weight:bold;color:#2b6cb0;text-decoration:underline">Passer à la Phase 3</a>